In [1]:
# Importing dependencies and libraries
import numpy as np
import json
import requests
import pandas as pd
import time

from config import api_key
from citipy import citipy
from IPython.display import clear_output

In [3]:
# Building the 500+ cities list

# Setting ranges to look for cities
latitude_range = np.random.randint(-90,91, 10) #change 10 to 70 when testing is done
longitude_range = np.random.randint(-180,181, 10) #change 10 to 70 when testing is done

# This loop will ensure that the sample size is enough
# At least 500 (will do 550 just to be sure) cities with data are needed, 
# with a 5.8% of missing data 577 cities is the minimum sample size
test = False
while test == False:

    cities_list = []
    #nested for loop that will retrieve every city
    for x in longitude_range:
        for y in latitude_range:
            city = citipy.nearest_city(x,y)
            cities_list.append(city.city_name)
    if len(cities_list) >= 0: #change conditional to 577 after testing
        test = True
#converting the resulting list to a dataframe, dropping the duplicates and reindexing
cities_list_df = pd.DataFrame({'Cities':cities_list})
cities_list_df = cities_list_df.drop_duplicates().reset_index(drop = True)
print(f'Sample size is {len(cities_list_df.index)} list built succesfully')
cities_list_df.head(10)

Sample size is 33 list built succesfully


,Cities
0,saint-philippe
1,talcahuano
2,arraial do cabo
3,jamestown
4,lebu
5,luderitz
6,port alfred
7,tsihombe
8,dikson
9,qaanaaq


In [13]:
#API requests

# Lists initialization
latitude = []
longitude = []
temperature = []
humidity = []
cloudiness = []
wind = []
failed_cities = []

# Base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Variable initialization for progress tracker
request_num = 0
succ_request = 0
failed_request = 0

# for loop to request data from all cities
for city in cities_list_df['Cities']:
    
    # Build query URL
    query_url = url + "appid=" + api_key + "&q=" + city + "&units=imperial"
    
    # Request data for the current city
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    # Printing progress information
    clear_output(wait=True)
    request_num += 1
    print(f'Processing request {request_num}/{len(cities_list_df.index)}')
    print(f'{succ_request} succesfull {failed_request} failed')
    
    # Making sure the request was succesful
    if weather_response.status_code == 200:
        # Appending current data to lists
        latitude.append(weather_json['coord']['lat'])
        longitude.append(weather_json['coord']['lon'])
        temperature.append(weather_json['main']['temp'])
        humidity.append(weather_json['main']['humidity'])
        cloudiness.append(weather_json['clouds']['all'])
        wind.append(weather_json['wind']['speed'])
        succ_request += 1
    else:
        failed_request += 1
        # Saving the failed city to be able to delete it from the dataframe
        failed_cities.append(city)
    
    # Making sure that time between requests is enough so that API key doesn't get blocked
    time.sleep(1.01)

# Print out final request results
clear_output(wait=True)   
print(f'Processing request {request_num}/{len(cities_list_df.index)}')
print(f'{succ_request} succesfull {failed_request} failed')
print('API requests finished')

Processing request 33/33
28 succesfull 5 failed
API requests finished


In [14]:
# Cleanind and building the results dataframe

clean_weather_df = cities_list_df.copy(deep=True)

# Deleting the cities with missing values
for city in failed_cities:
    # Finding the row to delete
    row_index = clean_weather_df.loc[clean_weather_df['Cities'] == city, :].index
    clean_weather_df.drop(row_index, inplace = True, axis = 0)

# Appending the data retrieved
clean_weather_df['Latitude'] = latitude
clean_weather_df['Longitude'] = longitude
clean_weather_df['Temperature (°F)'] = temperature
clean_weather_df['Humidity'] = humidity
clean_weather_df['Cloudiness'] = cloudiness
clean_weather_df['Wind Speed (mph)']= wind
clean_weather_df.head(10)

,Cities,Latitude,Longitude,Temperature (°F),Humidity,Cloudiness,Wind Speed (mph)
0,saint-philippe,-21.3585,55.7679,72.23,78,75,11.50
1,talcahuano,-36.7167,-73.1167,62.60,63,20,3.44
2,arraial do cabo,-22.9661,-42.0278,77.77,69,0,21.85
3,jamestown,42.0970,-79.2353,49.17,49,1,8.05
4,lebu,-37.6167,-73.6500,58.86,68,52,4.50
5,luderitz,-26.6481,15.1594,66.88,73,0,16.82
6,port alfred,-33.5906,26.8910,69.30,74,3,26.08
8,dikson,73.5069,80.5464,32.36,97,100,20.80
9,qaanaaq,77.4840,-69.3632,22.86,83,100,7.29
10,narsaq,60.9167,-46.0500,46.40,46,20,9.22


['tsihombe', 'barentsburg', 'amderma', 'illoqqortoormiut', 'belushya guba']